In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import re

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [3]:
df = pd.read_csv(
    "/Users/paniket/TU_Eindhoven/2_Study/Q2_JBI100_Visualisation_4/4_Code/JBI100_Visualisation/data/filtered_cleaned_shark_data.csv"
)
df.head(2)

,incident_month,incident_year,injury_severity,latitude,longitude,month_year,shark_behaviour_generic,shark_behaviour_specific,site_category_cleaned,state_names,victim_injury,location,provoked_unprovoked,victim_gender,victim_activity,shark_common_name,shark_scientific_name,no_sharks,reference,injury_location,data_source,shark_identification_method,shark_behaviour,victim_age
0,1,1791,major_lacerations,-33.86666666666667,151.2,1791-01-01,unknown,unknown,coastal,New South Wales,fatal,near sydney,unprovoked,female,swimming,white shark,Carcharodon carcharias,NaN,"shark&survl, whitley 1958, book ref 1793",torso,book,"bite analysis, shark behaviour, geographical location",unknown,NaN
1,3,1803,NaN,-25.833333333333332,113.88333333333334,1803-03-01,swam_at_victim,swam_at_victim,coastal,Western Australia,injured,"hamelin bay, faure island",unprovoked,male,swimming,tiger shark,Galeocerdo cuvier,1.0,"balgridge,green,taylor,whitley 1940",unknown,book,"bite analysis, shark behaviour, geographical location",swam at victim,NaN


In [4]:
# df["victim_injury"].unique()

In [5]:
# df.loc[df["victim_injury"]=="uninjured"]

In [6]:
shark_data = df.copy()

In [7]:
# df.columns

In [8]:
# df

In [9]:
# shark_attacks_over_time = (
#     shark_data.groupby("incident_year").size().reset_index(name="attack_count")
# )

# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=shark_attacks_over_time["incident_year"],
#         y=shark_attacks_over_time["attack_count"],
#         mode="lines",
#         line=dict(color="#00796b", width=2),
#         name="Shark Attacks",
#     )
# )

# fig.update_layout(
#     title="Shark Attacks Over Time (Years)",
#     xaxis_title="Year",
#     yaxis_title="Number of Attacks",
#     font=dict(color="#00796b", size=12),
#     plot_bgcolor="rgba(0,0,0,0)",
#     paper_bgcolor="rgba(0,0,0,0)",
#     xaxis=dict(showgrid=False),
#     yaxis=dict(showgrid=False),
# )

# # Show the graph
# fig.show()

In [10]:
# shark_attacks_by_injury = (
#     shark_data.groupby(["incident_year", "victim_injury"]).size().reset_index(name="attack_count")
# )

# # Initialize a new figure for multiple lines
# fig = go.Figure()

# # Define colors for each injury type
# injury_colors = {
#     "fatal": "#8d6e63",
#     "injured": "#5c6bc0",
#     "uninjured": "#26a69a",
#     "unknown": "#455a64",
# }

# # Add a line for each victim_injury type
# for injury_type, color in injury_colors.items():
#     filtered_data = shark_attacks_by_injury[shark_attacks_by_injury["victim_injury"] == injury_type]
#     fig.add_trace(
#         go.Scatter(
#             x=filtered_data["incident_year"],
#             y=filtered_data["attack_count"],
#             mode="lines",
#             line=dict(color=color, width=2),
#             name=injury_type.capitalize(),
#         )
#     )

# # Update layout for aesthetics
# fig.update_layout(
#     title="Shark Attacks Over Time by Type of Injury",
#     xaxis_title="Year",
#     yaxis_title="Number of Attacks",
#     font=dict(color="#00796b", size=12),
#     plot_bgcolor="rgba(0,0,0,0)",
#     paper_bgcolor="rgba(0,0,0,0)",
#     xaxis=dict(showgrid=False),
#     yaxis=dict(showgrid=False),
#     legend_title="Type of Injury",
# )

# # Show the graph
# fig.show()

In [11]:
incidents_per_month = (
    shark_data.groupby(["incident_month", "victim_injury"])
    .size()
    .reset_index(name="count")
)

months = list(range(1, 13))  # 1 to 12 for all months
month_names = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

incidents_per_month["month_name"] = incidents_per_month["incident_month"].map(
    dict(zip(months, month_names))
)

injury_types = ["fatal", "injured", "uninjured", "unknown"]
color_map = {
    "fatal": "#ab47bc",
    "injured": "#7e57c2",
    "uninjured": "#26a69a",
    "unknown": "#8d6e63",
}

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=[
        "Fatal Incidents",
        "Injured Incidents",
        "Uninjured Incidents",
        "Unknown Incidents",
    ],
    horizontal_spacing=0.1,
    vertical_spacing=0.25,
)

subplot_positions = {
    "fatal": (1, 1),
    "injured": (1, 2),
    "uninjured": (2, 1),
    "unknown": (2, 2),
}

for injury_type in injury_types:
    filtered_data = incidents_per_month[
        incidents_per_month["victim_injury"] == injury_type
    ]
    filtered_data = (
        filtered_data.set_index("month_name")
        .reindex(month_names, fill_value=0)
        .reset_index()
    )

    row, col = subplot_positions[injury_type]
    fig.add_trace(
        go.Bar(
            x=filtered_data["month_name"],
            y=filtered_data["count"],
            name=injury_type.capitalize(),
            marker_color=color_map[injury_type],
        ),
        row=row,
        col=col,
    )

fig.update_layout(
    title="Shark Incidents Per Month by Injury Type",
    font=dict(color="#00796b", size=14),
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    showlegend=False,
    height=900,
    width=1000,
)

fig.update_xaxes(title_text="Month", tickangle=-45, row=1, col=1)
fig.update_xaxes(title_text="Month", tickangle=-45, row=1, col=2)

fig.update_xaxes(title_text="Month", tickangle=-45, row=2, col=1)
fig.update_xaxes(title_text="Month", tickangle=-45, row=2, col=2)

fig.update_yaxes(title_text="Number of Incidents", row=1, col=1)
fig.update_yaxes(title_text="Number of Incidents", row=2, col=1)

# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)

# Show the graph
fig.show()

In [12]:
# Prepare the data: Group by incident month and victim injury
incidents_per_month = (
    shark_data.groupby(["incident_month", "victim_injury"])
    .size()
    .reset_index(name="count")
)

# Ensure all months are present
months = list(range(1, 13))  # 1 to 12 for all months
month_names = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

# Map month numbers to month names
incidents_per_month["month_name"] = incidents_per_month["incident_month"].map(
    dict(zip(months, month_names))
)

# Injury types and their corresponding colors
injury_types = ["fatal", "injured", "uninjured", "unknown"]
color_map = {
    "fatal": "#ab47bc",
    "injured": "#7e57c2",
    "uninjured": "#26a69a",
    "unknown": "#8d6e63",
}

# Create a subplot figure with 2 rows and 2 columns, increasing vertical spacing
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=[
        "Fatal Incidents",
        "Injured Incidents",
        "Uninjured Incidents",
        "Unknown Incidents",
    ],
    horizontal_spacing=0.1,
    vertical_spacing=0.25,  # Increased vertical spacing
)

# Map each injury type to a specific subplot
subplot_positions = {
    "fatal": (1, 1),
    "injured": (1, 2),
    "uninjured": (2, 1),
    "unknown": (2, 2),
}

for injury_type in injury_types:
    # Ensure all months are accounted for
    filtered_data = incidents_per_month[
        incidents_per_month["victim_injury"] == injury_type
    ]
    filtered_data = (
        filtered_data.set_index("month_name")
        .reindex(month_names, fill_value=0)
        .reset_index()
    )

    row, col = subplot_positions[injury_type]
    fig.add_trace(
        go.Bar(
            x=filtered_data["month_name"],
            y=filtered_data["count"],
            name=injury_type.capitalize(),
            marker_color=color_map[injury_type],
        ),
        row=row,
        col=col,
    )

# Update layout for aesthetics and size
fig.update_layout(
    title="Shark Incidents Per Month by Injury Type",
    font=dict(color="#00796b", size=14),
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent background
    paper_bgcolor="rgba(0,0,0,0)",
    showlegend=False,
    height=900,  # Increased height
    width=1000,  # Width remains the same
)

# Remove gridlines from all subplots and ensure consistent y-axis range
fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    showgrid=False, range=[0, 150]
)  # Set y-axis range from 0 to 150

# Update x-axis titles
fig.update_xaxes(title_text="Month", tickangle=-45, row=1, col=1)
fig.update_xaxes(title_text="Month", tickangle=-45, row=1, col=2)
fig.update_xaxes(title_text="Month", tickangle=-45, row=2, col=1)
fig.update_xaxes(title_text="Month", tickangle=-45, row=2, col=2)

# Update y-axis titles
fig.update_yaxes(title_text="Number of Incidents", row=1, col=1)
fig.update_yaxes(title_text="Number of Incidents", row=2, col=1)

# Show the graph
fig.show()

In [13]:
monthly_injury_data = (
    incidents_per_month.groupby(["month_name", "victim_injury"])["count"]
    .sum()
    .reset_index()
)

fig = px.bar(
    monthly_injury_data,
    x="month_name",
    y="count",
    color="victim_injury",
    barmode="group",  # Grouped bar mode
    color_discrete_map={
        "fatal": "#ab47bc",
        "injured": "#7e57c2",
        "uninjured": "#26a69a",
        "unknown": "#8d6e63",
    },
    title="Shark Incidents Per Month by Injury Type",
    labels={
        "month_name": "Month",
        "count": "Number of Incidents",
        "victim_injury": "Injury Type",
    },
    category_orders={"month_name": month_names},
)

fig.update_layout(
    font=dict(color="#00796b", size=14),
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent background
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis_title="Month",
    yaxis_title="Number of Incidents",
    legend_title="Injury Type",
    height=700,
    width=1050,
)

fig.show()

In [14]:
df.loc[df["victim_injury"] == "unknown"]

,incident_month,incident_year,injury_severity,latitude,longitude,month_year,shark_behaviour_generic,shark_behaviour_specific,site_category_cleaned,state_names,victim_injury,location,provoked_unprovoked,victim_gender,victim_activity,shark_common_name,shark_scientific_name,no_sharks,reference,injury_location,data_source,shark_identification_method,shark_behaviour,victim_age
492,4,1959,NaN,-33.95,151.250.,1959-04-01,unknown,unknown,coastal,New South Wales,unknown,"maroubra beach, sydney",unknown,unknown,unknown,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN


# Dangerous sharks


In [15]:
shark_counts = shark_data["shark_common_name"].value_counts().reset_index()
shark_counts.columns = ["shark_common_name", "count"]


top_5_sharks = shark_counts.head(7)
others = shark_counts.iloc[7:]
num_others = others["shark_common_name"].nunique()
others_count = others["count"].sum()
others_row = pd.DataFrame(
    [
        {
            "shark_common_name": f"others: {num_others} types",
            "count": others_count,
        }
    ]
)
top_5_sharks = pd.concat([top_5_sharks, others_row], ignore_index=True)

others_text = "<br>".join(
    [
        f"{row['shark_common_name']}: {row['count']} incidents"
        for _, row in others.iterrows()
    ]
)


top_5_sharks.loc[
    top_5_sharks["shark_common_name"] == f"others: {num_others} types",
    "hover_text",
] = others_text

top_5_sharks["hover_text"] = top_5_sharks["hover_text"].fillna(
    top_5_sharks["shark_common_name"]
    + ": "
    + top_5_sharks["count"].astype(str)
    + " incidents"
)

colors = [
    "#ab47bc",
    "#7e57c2",
    "#2e7d32",
    "#455a64",
    "#795548",
    "#303f9f",
    "#5e35b1",
    "#4e342e",
]

fig = px.bar(
    top_5_sharks,
    x="shark_common_name",
    y="count",
    title="Most Dangerous Sharks",
    labels={"shark_common_name": "Shark Type", "count": "Number of Incidents"},
    color="shark_common_name",
    color_discrete_sequence=colors,
    text="hover_text",
)

fig.update_traces(
    hovertemplate="%{text}",
    textposition="none",
)

fig.update_layout(
    font=dict(color="#00796b", size=14),
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis_title="Shark Type",
    yaxis_title="Number of Incidents",
    showlegend=False,
)

fig.show()

In [16]:
shark_victim_injury_counts = (
    shark_data.groupby(["shark_common_name", "victim_injury"])
    .size()
    .reset_index(name="count")
)
shark_victim_injury_counts["total_sum"] = 0
unique_sharks = shark_victim_injury_counts["shark_common_name"].unique()
for shark in unique_sharks:
    dfx = shark_victim_injury_counts.loc[
        shark_victim_injury_counts["shark_common_name"] == shark
    ]
    sum_incident = dfx["count"].sum()
    shark_victim_injury_counts.loc[
        shark_victim_injury_counts["shark_common_name"] == shark, "total_sum"
    ] = sum_incident
shark_victim_injury_counts = shark_victim_injury_counts.sort_values(
    "total_sum", ascending=False
)

shark_victim_injury_counts_top = shark_victim_injury_counts.loc[
    shark_victim_injury_counts["shark_common_name"].isin(
        shark_victim_injury_counts["shark_common_name"].unique()[:7]
    )
]
shark_victim_injury_counts_others = shark_victim_injury_counts.loc[
    ~shark_victim_injury_counts["shark_common_name"].isin(
        shark_victim_injury_counts["shark_common_name"].unique()[:7]
    )
]

# # hover stuff like in the previous graph, idk why i am doing this to myself.
# ye idc
# others_detailed_text = "<br>".join(
#     [
#         f"{row['shark_common_name']}: {row['count']} incidents"
#         for _, row in shark_victim_injury_counts_others.iterrows()
#     ]
# )
# shark_victim_injury_counts_others.loc[
#     shark_victim_injury_counts_others["shark_common_name"].str.contains(
#         "others"
#     ),
#     "hover_text",
# ] = others_detailed_text

# shark_victim_injury_counts_top["hover_text"] = np.nan
# shark_victim_injury_counts_top["hover_text"] = shark_victim_injury_counts_top[
#     "hover_text"
# ].fillna(
#     shark_victim_injury_counts_top["shark_common_name"]
#     + ": "
#     + shark_victim_injury_counts_top["count"].astype(str)
#     + " incidents"
# )


others_fatal_count = shark_victim_injury_counts_others.loc[
    shark_victim_injury_counts_others["victim_injury"] == "fatal"
]["count"].sum()
others_injured_count = shark_victim_injury_counts_others.loc[
    shark_victim_injury_counts_others["victim_injury"] == "injured"
]["count"].sum()
others_uninjured_count = shark_victim_injury_counts_others.loc[
    shark_victim_injury_counts_others["victim_injury"] == "uninjured"
]["count"].sum()
others_unknown_count = shark_victim_injury_counts_others.loc[
    shark_victim_injury_counts_others["victim_injury"] == "unknown"
]["count"].sum()
total_sum = shark_victim_injury_counts_others["count"].sum()


# others_fatal_hover = shark_victim_injury_counts_others.loc[
#     shark_victim_injury_counts_others["victim_injury"] == "fatal"
# ]["hover_text"].values
# others_injured_hover = shark_victim_injury_counts_others.loc[
#     shark_victim_injury_counts_others["victim_injury"] == "injured"
# ]["hover_text"].values
# others_uninjured_hover = shark_victim_injury_counts_others.loc[
#     shark_victim_injury_counts_others["victim_injury"] == "uninjured"
# ]["hover_text"].values
# others_unknown_hover = shark_victim_injury_counts_others.loc[
#     shark_victim_injury_counts_others["victim_injury"] == "unknown"
# ]["hover_text"].values

num_others = shark_victim_injury_counts_others["shark_common_name"].nunique()


shark_victim_injury_counts_others = pd.DataFrame(
    {
        "shark_common_name": [
            f"others: {num_others} types",
            f"others: {num_others} types",
            f"others: {num_others} types",
        ],
        "victim_injury": ["injured", "uninjured", "unknown"],
        "count": [
            others_injured_count,
            others_uninjured_count,
            others_unknown_count,
        ],
        "total_sum": [total_sum, total_sum, total_sum],
        # "hover_text": [others_injured_hover, others_uninjured_hover, others_unknown_hover],
    }
)
shark_victim_injury_counts = pd.concat(
    [shark_victim_injury_counts_top, shark_victim_injury_counts_others]
).reset_index(drop=True)
# shark_victim_injury_counts

In [17]:
shark_victim_injury_counts["total_sum_text"] = shark_victim_injury_counts[
    "total_sum"
].astype(str)

fig = px.bar(
    shark_victim_injury_counts,
    x="shark_common_name",
    y="count",
    color="victim_injury",
    title="Shark Incidents by Type of Injury",
    labels={
        "shark_common_name": "Shark Type",
        "count": "Number of Incidents",
        "victim_injury": "Type of Injury",
    },
    barmode="group",
    color_discrete_sequence=colors,
    # text="hover_text", # Commented out hover text stuff
)

fig.update_traces(textposition="none")
fig.update_layout(
    font=dict(color="#00796b", size=14),
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis_title="Shark Type",
    yaxis_title="Number of Incidents",
    showlegend=True,
    height=750,
    width=1000,
)

annotations = []
for index, row in (
    shark_victim_injury_counts.groupby("shark_common_name").first().iterrows()
):
    annotations.append(
        dict(
            x=row.name,
            y=-5,
            text=f"Total: {row['total_sum']}",
            showarrow=False,
            font=dict(color="#00796b", size=12),
            xanchor="center",
            yanchor="top",
        )
    )

fig.update_layout(annotations=annotations)

fig.show()

In [18]:
# df["victim_activity"].unique()

In [19]:
shark_monthly_activity = (
    shark_data.groupby(
        ["incident_month", "victim_activity", "provoked_unprovoked"]
    )
    .size()
    .reset_index(name="count")
)

# Add a "total_count" column which is the sum of "count" for each victim_activity
shark_monthly_activity["total_count"] = shark_monthly_activity.groupby(
    ["incident_month", "victim_activity"]
)["count"].transform("sum")
shark_monthly_activity

,incident_month,victim_activity,provoked_unprovoked,count,total_count
0,1,boarding,provoked,4,36
1,1,boarding,unprovoked,32,36
2,1,diving,provoked,8,11
3,1,diving,unprovoked,3,11
4,1,fishing,provoked,7,7
...,...,...,...,...,...
170,12,unknown,unknown,1,4
171,12,unknown,unprovoked,2,4
172,12,unmotorised boating,provoked,1,2
173,12,unmotorised boating,unprovoked,1,2


In [20]:
shark_activity = (
    shark_data.groupby(
        ["victim_activity", "provoked_unprovoked", "shark_common_name"]
    )
    .size()
    .reset_index(name="count")
)
shark_activity["total_count"] = shark_activity.groupby(
    ["victim_activity", "shark_common_name"]
)["count"].transform("sum")
shark_activity.sort_values(["total_count"], ascending=False)

,victim_activity,provoked_unprovoked,shark_common_name,count,total_count
4,boarding,provoked,white shark,3,180
15,boarding,unprovoked,white shark,177,180
115,swimming,unprovoked,bull shark,135,140
102,swimming,provoked,bull shark,5,140
122,swimming,unprovoked,tiger shark,94,98
...,...,...,...,...,...
80,spearfishing,provoked,blacktip reef shark,1,1
78,snorkelling,unprovoked,whitetip reef shark,1,1
75,snorkelling,unprovoked,unknown,1,1
71,snorkelling,unprovoked,dusky shark,1,1


In [21]:
shark_activity = (
    shark_data.groupby(["provoked_unprovoked", "shark_common_name"])
    .size()
    .reset_index(name="count")
)
shark_activity["total_count"] = shark_activity.groupby(["shark_common_name"])[
    "count"
].transform("sum")
shark_activity.sort_values(["total_count"], ascending=False)

,provoked_unprovoked,shark_common_name,count,total_count
43,unprovoked,white shark,307,371
27,unknown,white shark,1,371
21,provoked,white shark,63,371
18,provoked,tiger shark,104,230
40,unprovoked,tiger shark,126,230
23,provoked,wobbegong,100,203
28,unknown,wobbegong,1,203
45,unprovoked,wobbegong,102,203
4,provoked,bull shark,31,202
32,unprovoked,bull shark,171,202


In [22]:
shark_data

,incident_month,incident_year,injury_severity,latitude,longitude,month_year,shark_behaviour_generic,shark_behaviour_specific,site_category_cleaned,state_names,victim_injury,location,provoked_unprovoked,victim_gender,victim_activity,shark_common_name,shark_scientific_name,no_sharks,reference,injury_location,data_source,shark_identification_method,shark_behaviour,victim_age
0,1,1791,major_lacerations,-33.86666666666667,151.2,1791-01-01,unknown,unknown,coastal,New South Wales,fatal,near sydney,unprovoked,female,swimming,white shark,Carcharodon carcharias,NaN,"shark&survl, whitley 1958, book ref 1793",torso,book,"bite analysis, shark behaviour, geographical location",unknown,NaN
1,3,1803,NaN,-25.833333333333332,113.88333333333334,1803-03-01,swam_at_victim,swam_at_victim,coastal,Western Australia,injured,"hamelin bay, faure island",unprovoked,male,swimming,tiger shark,Galeocerdo cuvier,1.0,"balgridge,green,taylor,whitley 1940",unknown,book,"bite analysis, shark behaviour, geographical location",swam at victim,NaN
2,1,1807,minor_lacerations,-33.86666666666667,151.2,1807-01-01,bit_victim_on_wrist,bit_victim_on_wrist,estuary_harbour,New South Wales,injured,"cockle bay, sydney harbour",unprovoked,male,swimming,bull shark,Carcharhinus leucas,1.0,sydney gazette 18.1.1807,"arm, hand",media outlet,"bite analysis, shark behaviour",bit victim on wrist,NaN
3,1,1820,major_lacerations,-42.8,147.53333333333333,1820-01-01,bit_leg,bit_leg,coastal,Tasmania,fatal,"sweetwater point, pitt water",provoked,male,swimming,shark_not_known,shark_not_known,1.0,"shark&survl, c. black researcher",leg,witness account,unknown,bit victim on leg,NaN
4,1,1825,minor_lacerations,-33.85,151.21666666666667,1825-01-01,bit_leg,bit_leg,estuary_harbour,New South Wales,injured,"kirribili point, sydney harbour",unprovoked,male,swimming,bull shark,Carcharhinus leucas,1.0,maitland daily mercury 13.11.1899,leg,media outlet,"bite analysis, shark behaviour, geographical location",bit legs,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,12,2023,fatality,-34.679356,137.684906.,2023-12-01,unknown,unknown,coastal,South Australia,fatal,"ethel beach, yorke penninsula",unprovoked,male,surfing,white shark,Carcharodon carcharias,1.0,NaN,fatal,unknown,unknown,unknown,15.0
1229,1,2024,lacerations,-34.3333,135.7500.,2024-01-01,unknown,unknown,coastal,South Australia,injured,eyre penninsula,unprovoked,male,surfing,shark_not_known,shark_not_known,1.0,NaN,leg,unknown,unknown,unknown,64.0
1230,1,2024,NaN,-32.333,152.533,2024-01-01,unknown,unknown,estuary_harbour,New South Wales,injured,elizabeth bay,unprovoked,unknown,unknown,bull shark,Carcharhinus leucas,1.0,NaN,unknown,unknown,unknown,unknown,NaN
1231,2,2024,NaN,-30.30591,115.03825,2024-02-01,unknown,unknown,coastal,Western Australia,injured,jurien bay,unprovoked,unknown,unknown,tiger shark,Galeocerdo cuvier,1.0,NaN,unknown,unknown,unknown,unknown,NaN
